In [ ]:
from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cftime
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

# Raster Plotting

So far the plotting we have done has been to draw contours or color-filled contours, which both attempt to separate the data by putting higher values on one side of the contour and lower on the other. Matplotlib uses an algorith to find these separation points to be able to plot our data. And contours are great, but sometimes its just not the right fit for our data, or we wish to represent the gridded data in a more authentic raster format.

Our gridded data is a version of what we call 'raster' data, which is "any pixelated (or gridded) data where each pixel is associated with a specific geographical location" (https://datacarpentry.org/organization-geospatial/01-intro-raster-data/). There are times when you may wish to represent the data distinctly as there individual grids and color them accordingly. This could be what you want for any form of gridded including satellite imagery. There are two main functions from Matplotlib that we could use, but we'll just focus on one for now, `pcolormesh`.

But first, let's read in some data so we have something to plot. Here we are going to read in psuedo-surface air temperature data from the NCEP/NCAR Reanalysis dataset. (This data comes from a sigma surface near the ground, which is how we get the `sig995` in the name of the file.) We'll subset for a days in 2021 via the Xarray `.sel` method and using the datetime module to specify a data and time.

Here is a link to a THREDDS server hosted by NOAA that contains a number of different types of datasets.

https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html

Navigate to the `ncep.reanalysis` folder, then to the `surface` folder and select the `air.sig995.2021.nc` file. There are four different methods to access this data file and we want to use OPENDAP. Click that link and you'll be brought to a page that describes what is contained in the data file. What we are most interested in is the link at the top of the page in the box labeled Data URL. Copy that link to be able to use Xarray to read this data file remotely. 

In [ ]:
date = datetime(2021, 3, 8, 12)

# Open Dataset
ds_air = xr.open_dataset('')
ds_air = ds_air.sel(time=date)

## Color-fill Raster Imagery
The Matplotlib function to fill in each grid cell with a different color is `pcolormesh`, which works similarly to our `contour` and `contourf` functions.

```python
ax.pcolormesh(x, y, Z, vmin=<minimum_value>, vmax=<maximum_value>, cmap=<colormap>, shading='auto')
```

The key difference here is that the levels are determined by setting two keyword arguments: `vmin` and `vmax`. These values are both inclusive for plotting meaning if you wanted to plot values from -50 to 50 you would set `vmin=-50` and `vmax=50`.  Additionally, you will always want to include the `shading='auto'` keyword arugment (as exampled above) to fill our grid cells appropriately.

In [ ]:
plt.figure(1, figsize=(15, 15))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

cf = ax.pcolormesh()

plt.colorbar(cf, ax=ax, orientation='horizontal', pad=0, aspect=50)
ax.coastlines()

plt.show()

Note that with this type of plotting (and due to our `shading='auto'` setting, we don't have the missing line of color around the Prime Meridian (0E). A nice little bonus for plotting our raster data instead of contouring.

## Compute Anomaly Value
Now that we have a raster-filled plot, let's take a different look at our data through computing an anomaly value.

A value can be broken up into two parts, a mean and its anomaly.

$a = \bar{a} + a'$

where $\bar{a}$ is the mean value and $a'$ is the anomaly value.

To solve for the anomaly value we add it to both sides and subtract our actual value from both sides to get

$a' = a - \bar{a}$

If we want to compute the temperature anomaly values that we plotted above, we would need to read in (or compute) a long term mean (LTM) dataset. Lucky for us, NOAA provides such a file. We can read in the corresponding LTM data set, where the means are computed from the 1981-2010 values. There are multiple different files that contain this data, but what we want is the one that corresponds to the every six hour data that we have already read in, so we can subtract the mean value for that exact day and time combination.

This is now "derived" data, meaning that it has been computed from other data and not a raw value.

Here is a link to a THREDDS server hosted by NOAA that contains a number of different types of datasets.

https://psl.noaa.gov/thredds/catalog/Datasets/catalog.html

Navigate to the `ncep.reanalysis.derived` folder, then to the `surface` folder and select the `air.sig995.4Xday.1981-2010.ltm.nc` file. There are four different methods to access this data file and we want to use OPENDAP. Click that link and you'll be brought to a page that describes what is contained in the data file. What we are most interested in is the link at the top of the page in the box labeled Data URL. Copy that link to be able to use Xarray to read this data file remotely. 

In [ ]:
ds_ltm = xr.open_dataset('https://psl.noaa.gov/thredds/dodsC/Datasets/ncep.reanalysis.derived/surface/air.sig995.4Xday.1981-2010.ltm.nc')

### cftime
The nature of long term mean (LTM) files are different from our regular daily value files. Here time is not associated with an individual day, but rather a set of 30 years (or some other set). Xarray therefore has difficulty interpreting the dates in the files and uses a different moduel, `cftime` to parse the time instead of Numpy. In order to access and subset our data we'll need to use that module as well.

Let's begin by taking a look at what the time variable looks like from our file.

Note how the time objects are `cftime.DatetimeGregorian` and are being referenced with year 1, but otherwise, the month, day, and hours are all as we would expect with a normal `datetime` object. Therefore, all we need is to select our time using the `cftime.DatetimeGregorian` function.

```python
ds = ds.sel(time=cftime.DatetimeGregorian(1, 5, 12, 18))
```

Assuming a standard dataset `ds` and using the default year 1 and specifying the month, day, and hour that we want to access.

In [ ]:
ds_ltm = ds_ltm.sel(time=)

We can plot the LTM values to see what they look like in the same way we would plot the values from a given date/time.

Now that we have subset our data, let's actually compute the anomaly value. We need to do this on the actualy data arrays, so we would want to use `ds_air.air` for our actual values and `ds_ltm.air` for our LTM values.

In [ ]:
anomaly = 

We now have a new Xarray DataArray that contains our coordinate variables along with the computed anomaly values, still called `air` within the new DataArray.

Now let's plot the anomaly values to see where on this particular day it was warmer and colder than the long term mean. To do this we want to use a diverging colormap (e.g., `plt.cm.bwr`) and set our `vmin` and `vmax` to be equal and opposite to place zero right in the center of the range.

## Compute and Plot Max/Min Values

The where method from an Xarray DataArray makes it easy to find our maximum and minimum values over the entire dataset.

In [ ]:
max_anom = anomaly.where(anomaly == anomaly.max(), drop=True)[0][0]
min_anom = anomaly.where(anomaly == anomaly.min(), drop=True)[0][0]

But how do we plot these values?

## Plotting Max/Min Values with Transform

To plot a marker at a given location we can use the `scatter` method:

```python
ax.scatter(x, y, s, marker='d', transform=ccrs.PlateCarree())
```
Here x is the x-coordinate location, y is the y-coordinate location, s is the size of the marker, and the keyword marker can be set to any of the valid Matplotlib markers.

Matplotlib Scatter Documentation: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html

Matplotlib Markers: https://matplotlib.org/stable/api/markers_api.html

As long as your x and y values are latitude/longitude coordinate values, then you would want to use the `ccrs.PlateCarree()` projection for your transform keyword argument.

To plot text at a given location on the plot we can use the `text` method:

```python
ax.text(x, y, string, transform=ccrs.PlateCarree())
```
Here x is the x-coordinate location, y is the y-coordinate location, and string is the text you wish to plot at the given point.

Matplotlib Text Documentation: https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.text.html

Again, as long as your x and y values are latitude/longitude coordinate values, then you would want to use the `ccrs.PlateCarree()` projection for your transform keyword argument.

Let's plot a black circle at the location of the maximum anomaly and label it Max and do the same for the minimum anomaly.

In [ ]:
plt.figure(1, figsize=(15, 15))
ax = plt.subplot(111, projection=ccrs.PlateCarree())

cf = ax.pcolormesh(ds_air.lon, ds_air.lat, anomaly,
                   vmin=-30, vmax=30, cmap=plt.cm.bwr, shading='auto')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50)

# Add Maximum Value Marker and Text
ax.scatter()
ax.text()

# Add Minimum Value Marker and Text
ax.scatter()
ax.text()

ax.coastlines()

gl = ax.gridlines(draw_labels=True)
gl.bottom_labels = False

plt.show()

## Exercise #1
Subset to a domain that only includes North America, plot the anomaly and find the maximum and minimum values over just that domain and plot them on the anomaly map.